<a href="https://colab.research.google.com/github/ogut77/DataScience/blob/main/Homework6V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Intro
Term deposits are a major source of income for a bank. A term deposit is a cash investment held at a financial institution. Your money is invested for an agreed rate of interest over a fixed amount of time, or term. The bank has various outreach plans to sell term deposits to their customers such as email marketing, advertisements, telephonic marketing, and digital marketing.

Telephonic marketing campaigns still remain one of the most effective way to reach out to people. However, they require huge investment as large call centers are hired to actually execute these campaigns. Hence, it is crucial to identify the customers most likely to convert beforehand so that they can be specifically targeted via call.

The data is related to direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe to a term deposit (variable y).

Content
The data is related to the direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed by the customer or not. The data folder contains two datasets:-

Bank.csv: 45,211 rows and 18 columns ordered by date (from May 2008 to November 2010)

Detailed Column Descriptions
bank client data:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
"blue-collar","self-employed","retired","technician","services")

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

6 - balance: average yearly balance, in euros (numeric)

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")
# related with the last contact of the current campaign:
9 - contact: contact communication type (categorical: "unknown","telephone","cellular")
10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

# other attributes:
13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)

15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

Output variable (desired target):

17 - y - has the client subscribed a term deposit? (binary: "yes","no")

Missing Attribute Values: None


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('https://raw.githubusercontent.com/ogut77/DataScience/main/data/Bank.csv',sep = ';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [37]:
import optuna
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


In [38]:
print(df.shape)
df.info()
df.isnull().sum()

(45211, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [39]:
#For object check the data
for cn in df.columns:
  if(df[cn].dtype==object):
    print(df[cn].value_counts())


job
blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: count, dtype: int64
marital
married     27214
single      12790
divorced     5207
Name: count, dtype: int64
education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64
default
no     44396
yes      815
Name: count, dtype: int64
housing
yes    25130
no     20081
Name: count, dtype: int64
loan
no     37967
yes     7244
Name: count, dtype: int64
contact
cellular     29285
unknown      13020
telephone     2906
Name: count, dtype: int64
month
may    13766
jul     6895
aug     6247
jun     5341
nov     3970
apr     2932
feb     2649
jan     1403
oct      738
sep      579
mar      477
dec      214
Name: count, dtype: int64
poutcome
unknown    36959
failure     4901
other  

In [40]:
# Following function converts non-numeric variables (e.g., 'category', 'object') into numeric using label encoding
# Note:
#Label encoding converts categorical values into integer codes. Each unique category is assigned a distinct number, such as 1, 2, 3, 4, etc.
#Example : In our data, education variable has following values:{ 'primary' 'secondary', 'tertiary', 'unknown'}
#and it is mapped { 'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3}

#  education  education_encoded
#  secondary                  1
#   tertiary                  2
#    primary                  0
#    unknown                  3
#   tertiary                  2
#    primary                  0
#
#Label encoding is generally suitable for tree-based models (e.g., decision trees, random forests, boosting methods).
#However, it may not be appropriate for models where the objective function relies on distance-based calculations, such as neural networks, support vector machines (SVM), or linear regression.
#For nominal features (categories with no intrinsic ordering, e.g., "red," "blue," "green"), label encoding can mislead the model by implying an ordinal relationship where none exists.
#In such cases, one-hot encoding is usually preferred.
#One-hot encoding creates binary columns (dummy variables) for each category in a categorical feature, avoiding the introduction of unintended ordinality.

def Encoder(df):
          from sklearn import preprocessing
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = preprocessing.LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


In [41]:
df=Encoder(df)
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,0


In [42]:
y = df['y'] #Output
X = df.drop('y',axis=1)
X

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3


In [43]:
from sklearn.model_selection import  train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)




Q1)Using  Random Forest,XGBoost, Light GBM and Gradient Boosting Classifier with default parameters (no parameter specifications except random_state) calculate Accuracy on Test data. Which method gives the best accuracy on test data

In [44]:

# Initialize classifiers with random_state
models = {
    "Random Forest": RandomForestClassifier(random_state=17),
    "Gradient Boosting": GradientBoostingClassifier(random_state=17),
    "XGBoost": XGBClassifier(random_state=17),
    "LightGBM": LGBMClassifier(random_state=17)
}

# Train and evaluate each model
accuracy_results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_results[name] = accuracy_score(y_test, y_pred)

# Display accuracy for each model
for model_name, acc in accuracy_results.items():
    print(f"{model_name}: Accuracy = {acc:.4f}")


[LightGBM] [Info] Number of positive: 3940, number of negative: 29968
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 33908, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116197 -> initscore=-2.028949
[LightGBM] [Info] Start training from score -2.028949
Random Forest: Accuracy = 0.9025
Gradient Boosting: Accuracy = 0.9019
XGBoost: Accuracy = 0.9037
LightGBM: Accuracy = 0.9074


LightGBM is better 


Q2) Using optuna hyperparmeter optimization technique and 100 trial

 a)find best methods with  parameters  using Cross validation (CV=3) technique for the range of   parameters below. What are the best parameters for the method with highest cross validation accuracy?
 For random forest


  "max_depth"   : trial.suggest_int("max_depth", 2,  X_train.shape[1]),
  "max_features": trial.suggest_int("max_features", 2, X_train.shape[1])

For XGBoost, Light GBM and Gradient Boosting Classifier

  "max_depth": trial.suggest_int("max_depth", 2, X_train.shape[1]),
  "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3,log=True)

where X_train.shape[1] is number of columnns in the train data.

 b)Evaluate the performance of the  method with highest cross validation accuracy on test data.What is the accuracy value? Are there any improvement of the same method with default parameters?


In [45]:
#a
n_features = X_train.shape[1]

def rf_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "max_features": trial.suggest_int("max_features", 2, n_features),
        "n_estimators": 100,
        "random_state": 42
    }
    model = RandomForestClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy").mean()

def xgb_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "use_label_encoder": False,
        "eval_metric": "logloss",
        "random_state": 42
    }
    model = XGBClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy").mean()

def lgbm_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "random_state": 42
    }
    model = LGBMClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy").mean()

def gb_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "random_state": 42
    }
    model = GradientBoostingClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy").mean()

# Run studies
def run_study(objective_func, name):
    study = optuna.create_study(direction="maximize", study_name=name)
    study.optimize(objective_func, n_trials=100)
    return study

rf_study = run_study(rf_objective, "RandomForest")


[I 2025-04-15 15:39:48,926] A new study created in memory with name: RandomForest
[I 2025-04-15 15:39:59,313] Trial 0 finished with value: 0.9035626435580686 and parameters: {'max_depth': 7, 'max_features': 13}. Best is trial 0 with value: 0.9035626435580686.
[I 2025-04-15 15:40:07,370] Trial 1 finished with value: 0.9009968538787275 and parameters: {'max_depth': 5, 'max_features': 16}. Best is trial 0 with value: 0.9035626435580686.
[I 2025-04-15 15:40:20,585] Trial 2 finished with value: 0.9039460173833117 and parameters: {'max_depth': 15, 'max_features': 10}. Best is trial 2 with value: 0.9039460173833117.
[I 2025-04-15 15:40:28,483] Trial 3 finished with value: 0.9035921446910912 and parameters: {'max_depth': 7, 'max_features': 10}. Best is trial 2 with value: 0.9039460173833117.
[I 2025-04-15 15:40:45,482] Trial 4 finished with value: 0.9037100683338194 and parameters: {'max_depth': 9, 'max_features': 15}. Best is trial 2 with value: 0.9039460173833117.
[I 2025-04-15 15:40:50,723]

In [46]:
xgb_study = run_study(xgb_objective, "XGBoost")


[I 2025-04-15 15:56:04,360] A new study created in memory with name: XGBoost
C:\Users\Gulsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:56:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Gulsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:56:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Gulsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca

In [47]:
lgbm_study = run_study(lgbm_objective, "LightGBM")

[I 2025-04-15 15:57:27,936] A new study created in memory with name: LightGBM


[LightGBM] [Info] Number of positive: 2627, number of negative: 19978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 22605, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116213 -> initscore=-2.028789
[LightGBM] [Info] Start training from score -2.028789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-04-15 15:57:28,295] Trial 0 finished with value: 0.9058629934922316 and parameters: {'max_depth': 4, 'learning_rate': 0.09700048944379512}. Best is trial 0 with value: 0.9058629934922316.


[LightGBM] [Info] Number of positive: 2627, number of negative: 19979
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 22606, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116208 -> initscore=-2.028839
[LightGBM] [Info] Start training from score -2.028839
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2025-04-15 15:57:28,689] Trial 1 finished with value: 0.8839801901640008 and parameters: {'max_depth': 5, 'learning_rate': 0.008366693937247472}. Best is trial 0 with value: 0.9058629934922316.


...

[I 2025-04-15 15:58:13,897] Trial 97 finished with value: 0.9077504293281153 and parameters: {'max_depth': 14, 'learning_rate': 0.04742560159006018}. Best is trial 51 with value: 0.9085761844633152.


[LightGBM] [Info] Number of positive: 2627, number of negative: 19978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 22605, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116213 -> initscore=-2.028789
[LightGBM] [Info] Start training from score -2.028789
[LightGBM] [Info] Number of positive: 2626, number of negative: 19979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001269 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 980
[LightGBM] [Info] Number of data points in the train set: 22605, number of used features: 16
[LightGBM] [Info] [bin

[I 2025-04-15 15:58:14,403] Trial 98 finished with value: 0.9060103869559715 and parameters: {'max_depth': 15, 'learning_rate': 0.02784334846219803}. Best is trial 51 with value: 0.9085761844633152.


[LightGBM] [Info] Number of positive: 2627, number of negative: 19979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 22606, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116208 -> initscore=-2.028839
[LightGBM] [Info] Start training from score -2.028839
[LightGBM] [Info] Number of positive: 2627, number of negative: 19978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001063 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 22605, number of used features: 16
[LightGBM] [Info] [bin

[I 2025-04-15 15:58:14,963] Trial 99 finished with value: 0.9060988616523623 and parameters: {'max_depth': 16, 'learning_rate': 0.033253212330402765}. Best is trial 51 with value: 0.9085761844633152.


In [48]:
gb_study = run_study(gb_objective, "GradientBoosting")


[I 2025-04-15 15:58:14,973] A new study created in memory with name: GradientBoosting
[I 2025-04-15 15:58:28,711] Trial 0 finished with value: 0.8838032329432165 and parameters: {'max_depth': 5, 'learning_rate': 0.002350553667233454}. Best is trial 0 with value: 0.8838032329432165.
[I 2025-04-15 15:58:59,116] Trial 1 finished with value: 0.8987555140386122 and parameters: {'max_depth': 9, 'learning_rate': 0.2931721688573279}. Best is trial 1 with value: 0.8987555140386122.
[I 2025-04-15 16:00:30,633] Trial 2 finished with value: 0.9010263106530679 and parameters: {'max_depth': 11, 'learning_rate': 0.10347205874876311}. Best is trial 2 with value: 0.9010263106530679.
[I 2025-04-15 16:01:02,118] Trial 3 finished with value: 0.9055975380910484 and parameters: {'max_depth': 5, 'learning_rate': 0.14867755443126732}. Best is trial 3 with value: 0.9055975380910484.
[I 2025-04-15 16:01:34,754] Trial 4 finished with value: 0.9024714356310196 and parameters: {'max_depth': 5, 'learning_rate': 0.0

In [49]:
# Find best model by accuracy
studies = {
    "RandomForest": rf_study,
    "XGBoost": xgb_study,
    "LightGBM": lgbm_study,
    "GradientBoosting": gb_study
}

best_model_name = max(studies, key=lambda name: studies[name].best_value)
best_study = studies[best_model_name]
best_params = best_study.best_params
best_cv_accuracy = best_study.best_value
print(f"Best model: {best_model_name}")
print(f"Best parameters: {best_params}")
print(f"Best CV accuracy: {best_cv_accuracy:.4f}")

Best model: LightGBM
Best parameters: {'max_depth': 11, 'learning_rate': 0.06733966863133908}
Best CV accuracy: 0.9086


In [50]:
#b
# Define model with best hyperparameters
def get_model(name, params):
    params.update({"n_estimators": 100, "random_state": 42})
    if name == "RandomForest":
        return RandomForestClassifier(**params)
    elif name == "XGBoost":
        params.update({"use_label_encoder": False, "eval_metric": "logloss"})
        return XGBClassifier(**params)
    elif name == "LightGBM":
        return LGBMClassifier(**params)
    elif name == "GradientBoosting":
        return GradientBoostingClassifier(**params)

# Best tuned model
best_model = get_model(best_model_name, best_params)
best_model.fit(X_train, y_train)
test_accuracy_tuned = accuracy_score(y_test, best_model.predict(X_test))

# Compare with default model
default_model = get_model(best_model_name, {})
default_model.fit(X_train, y_train)
test_accuracy_default = accuracy_score(y_test, default_model.predict(X_test))

print(f"Best model: {best_model_name}")
print(f"Best hyperparameters: {best_params}")
print(f"Cross-validation accuracy: {best_cv_accuracy:.4f}")
print(f"Test accuracy (tuned): {test_accuracy_tuned:.4f}")
print(f"Test accuracy (default): {test_accuracy_default:.4f}")


[LightGBM] [Info] Number of positive: 3940, number of negative: 29968
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005490 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 33908, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116197 -> initscore=-2.028949
[LightGBM] [Info] Start training from score -2.028949
[LightGBM] [Info] Number of positive: 3940, number of negative: 29968
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 33908, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116197 -> initscore=-2.028949
[Light

For Q3 and Q4 ,use the following data.

In [51]:
dr=pd.read_csv('https://raw.githubusercontent.com/ogut77/DataScience/main/data/diamond.csv')
dr

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171
...,...,...,...,...,...,...,...,...
5995,1.03,Ideal,D,SI1,EX,EX,GIA,6250
5996,1.00,Very Good,D,SI1,VG,VG,GIA,5328
5997,1.02,Ideal,D,SI1,EX,EX,GIA,6157
5998,1.27,Signature-Ideal,G,VS1,EX,EX,GIA,11206


In [52]:
def Encoder(df):
          from sklearn import preprocessing
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = preprocessing.LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


In [53]:
dr=Encoder(dr)
dr

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,2,4,2,3,0,1,5169
1,0.83,2,4,3,2,2,0,3470
2,0.85,2,4,2,0,0,1,3183
3,0.91,2,1,2,3,3,1,4370
4,0.83,2,3,2,0,0,1,3171
...,...,...,...,...,...,...,...,...
5995,1.03,2,0,2,0,0,1,6250
5996,1.00,4,0,2,3,3,1,5328
5997,1.02,2,0,2,0,0,1,6157
5998,1.27,3,3,3,0,0,1,11206


In [54]:
y = dr['Price'] #Output
X = dr.drop('Price',axis=1)
X

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report
0,1.10,2,4,2,3,0,1
1,0.83,2,4,3,2,2,0
2,0.85,2,4,2,0,0,1
3,0.91,2,1,2,3,3,1
4,0.83,2,3,2,0,0,1
...,...,...,...,...,...,...,...
5995,1.03,2,0,2,0,0,1
5996,1.00,4,0,2,3,3,1
5997,1.02,2,0,2,0,0,1
5998,1.27,3,3,3,0,0,1


In [55]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
import optuna
from sklearn.model_selection import cross_val_score

Q3)Using Linear Regression,Decison Tree Random Forest,XGBoost, Light GBM and Gradient Boosting Classifier with default parameters (no parameter specifications except random_state) calculate R2 statistics on test data. Which method gives the best accuracy on test data

In [57]:
models = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

r2_scores = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    r2_scores[name] = r2
    print(f"{name} R²: {r2:.4f}")

# Find best model
best_model_name = max(r2_scores, key=r2_scores.get)
print(f"\nBest model based on default parameters: {best_model_name} with R² = {r2_scores[best_model_name]:.4f}")

LinearRegression R²: 0.8234
DecisionTree R²: 0.9588
RandomForest R²: 0.9807
XGBoost R²: 0.9809
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 4500, number of used features: 7
[LightGBM] [Info] Start training from score 11827.946667
LightGBM R²: 0.9813
GradientBoosting R²: 0.9739

Best model based on default parameters: LightGBM with R² = 0.9813


Q4) Using optuna hyperparmeter optimization technique (100 trial)  with Random Forest,XGBoost, Light GBM and Gradient Boosting Regressor


a)find best methods with  parameters  using Cross validation (CV=3) technique for the range of   parameters below. What are the best parameters for the method with highest cross validation R2?

For random forest


  "max_depth"   : trial.suggest_int("max_depth", 2,  X_train.shape[1]),
  "max_features": trial.suggest_int("max_features", 2, X_train.shape[1])

For XGBoost, Light GBM and Gradient Boosting Classifier

  "max_depth": trial.suggest_int("max_depth", 2, X_train.shape[1]),
  "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3,log=True)

where X_train.shape[1] is number of columnns in the train data.

 b)Evaluate the performance of the  method with highest cross validation R2 on test data. What is the R2 value? Are there any improvement of the same method with default parameters?


In [58]:
#a
n_features = X_train.shape[1]

def rf_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "max_features": trial.suggest_int("max_features", 2, n_features),
        "n_estimators": 100,
        "random_state": 42
    }
    model = RandomForestRegressor(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="r2").mean()

def xgb_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "random_state": 42
    }
    model = XGBRegressor(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="r2").mean()

def lgbm_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "random_state": 42
    }
    model = LGBMRegressor(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="r2").mean()

def gb_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, n_features),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "n_estimators": 100,
        "random_state": 42
    }
    model = GradientBoostingRegressor(**params)
    return cross_val_score(model, X_train, y_train, cv=3, scoring="r2").mean()

# Run Optuna studies
def run_study(objective_func, name):
    study = optuna.create_study(direction="maximize", study_name=name)
    study.optimize(objective_func, n_trials=100)
    return study

rf_study = run_study(rf_objective, "RandomForestRegressor")


[I 2025-04-15 21:53:47,013] A new study created in memory with name: RandomForestRegressor
[I 2025-04-15 21:53:47,331] Trial 0 finished with value: 0.7948719878863658 and parameters: {'max_depth': 4, 'max_features': 2}. Best is trial 0 with value: 0.7948719878863658.
[I 2025-04-15 21:53:47,602] Trial 1 finished with value: 0.5632525526916333 and parameters: {'max_depth': 2, 'max_features': 2}. Best is trial 0 with value: 0.7948719878863658.
[I 2025-04-15 21:53:47,870] Trial 2 finished with value: 0.5632525526916333 and parameters: {'max_depth': 2, 'max_features': 2}. Best is trial 0 with value: 0.7948719878863658.
[I 2025-04-15 21:53:48,531] Trial 3 finished with value: 0.9610298378466339 and parameters: {'max_depth': 6, 'max_features': 7}. Best is trial 3 with value: 0.9610298378466339.
[I 2025-04-15 21:53:49,178] Trial 4 finished with value: 0.9718580954876749 and parameters: {'max_depth': 7, 'max_features': 6}. Best is trial 4 with value: 0.9718580954876749.
[I 2025-04-15 21:53:49,4

In [ ]:
xgb_study = run_study(xgb_objective, "XGBRegressor")


In [ ]:
lgbm_study = run_study(lgbm_objective, "LGBMRegressor")


In [ ]:
gb_study = run_study(gb_objective, "GradientBoostingRegressor")


In [ ]:
# Compare best R² scores
studies = {
    "RandomForest": rf_study,
    "XGBoost": xgb_study,
    "LightGBM": lgbm_study,
    "GradientBoosting": gb_study
}

best_model_name = max(studies, key=lambda name: studies[name].best_value)
best_study = studies[best_model_name]
best_params = best_study.best_params
best_cv_r2 = best_study.best_value

print(f"\nBest Model: {best_model_name}")
print(f"Best Hyperparameters: {best_params}")
print(f"Best Cross-Validation R²: {best_cv_r2:.4f}")

In [59]:
#b
def get_regressor(name, params):
    params.update({"n_estimators": 100, "random_state": 42})
    if name == "RandomForest":
        return RandomForestRegressor(**params)
    elif name == "XGBoost":
        return XGBRegressor(**params)
    elif name == "LightGBM":
        return LGBMRegressor(**params)
    elif name == "GradientBoosting":
        return GradientBoostingRegressor(**params)

# Best tuned model
best_model = get_regressor(best_model_name, best_params)
best_model.fit(X_train, y_train)
test_r2_tuned = r2_score(y_test, best_model.predict(X_test))

# Compare with default version of same model
default_model = get_regressor(best_model_name, {})
default_model.fit(X_train, y_train)
test_r2_default = r2_score(y_test, default_model.predict(X_test))

print(f"Test R² (tuned): {test_r2_tuned:.4f}")
print(f"Test R² (default): {test_r2_default:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 4500, number of used features: 7
[LightGBM] [Info] Start training from score 11827.946667
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 4500, number of used features: 7
[LightGBM] [Info] Start training from score 11827.946667
Test R² (tuned): 0.9822
Te